In [3]:
from common_import import *
result_dict = readpkl('result_dict.pkl')

In [ ]:
checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor',
    'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_with_Rz_input_centering_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678',
]

### H36M S15678 TRAIN

In [2]:
import prettytable

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']

checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering',
]

# 데이터를 리스트에 수집
data = []
for key in checkpoint_list:
    for subset in result_dict[key].keys():
        if 'STEP' in subset: continue
        if 'TRAIN_UNIV' in subset: continue
        splited = subset.split('-')
        if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
            dataset = splited[0]
        else:
            dataset = splited[0] + '-' + splited[-1]
        #print(dataset, type(dataset))
        e1 = float(result_dict[key][subset]['e1'])
        e2 = float(result_dict[key][subset]['e2'])
        data.append([dataset, e1, e2, key, subset])

# 데이터를 Subset 1순위, E1 2순위로 정렬
data_sorted = sorted(data, key=lambda x: (x[0], x[1]))

prev_dataset = ''
for i, row in enumerate(data_sorted):
    if prev_dataset != '' and prev_dataset != row[0]:
        data_sorted[i-1].append(True)
    else:
        data_sorted[i-1].append(False)
    prev_dataset = row[0]

# 정렬된 데이터를 테이블에 추가
for row in data_sorted:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

# 테이블 출력
print(pt)


+----------------------+--------+--------+---------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                    Checkpoint                                   |                                  Subset                                 |
+----------------------+--------+--------+---------------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 70.31  | 56.37  | MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN  |
| 3DHP-TEST_ALL_TRAIN  | 70.31  | 56.37  | MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN  |
| 3DH

### H36M S1 TRAIN

In [4]:
import prettytable

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']

checkpoint_list = [
    'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678',
    'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678',
    #'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678',
]

# 데이터를 리스트에 수집
data = []
for key in checkpoint_list:
    for subset in result_dict[key].keys():
        if 'STEP' in subset: continue
        if 'TRAIN_UNIV' in subset: continue
        splited = subset.split('-')
        if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
            dataset = splited[0]
        else:
            dataset = splited[0] + '-' + splited[-1]
        #print(dataset, type(dataset))
        e1 = float(result_dict[key][subset]['e1'])
        e2 = float(result_dict[key][subset]['e2'])
        data.append([dataset, e1, e2, key, subset])

# 데이터를 Subset 1순위, E1 2순위로 정렬
data_sorted = sorted(data, key=lambda x: (x[0], x[1]))

prev_dataset = ''
for i, row in enumerate(data_sorted):
    if prev_dataset != '' and prev_dataset != row[0]:
        data_sorted[i-1].append(True)
    else:
        data_sorted[i-1].append(False)
    prev_dataset = row[0]

# 정렬된 데이터를 테이블에 추가
for row in data_sorted:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

# 테이블 출력
print(pt)



+----------------------+--------+--------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
|       Dataset        |   E1   |   E2   |                                           Checkpoint                                           |                                  Subset                                 |
+----------------------+--------+--------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN  | 86.53  | 65.30  | MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678 |  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN  |
| 3DHP-TEST_ALL_TRAIN  | 86.53  | 65.30  | MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678 |  3DH

### FIT3D TR_S03

In [4]:
import prettytable

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']

checkpoint_list = [
    'MB_train_fit3d_gt_cam_no_factor_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03',
]

# 데이터를 리스트에 수집
data = []
for key in checkpoint_list:
    for subset in result_dict[key].keys():
        if 'STEP' in subset: continue
        if 'TRAIN_UNIV' in subset: continue
        splited = subset.split('-')
        if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
            dataset = splited[0]
        else:
            dataset = splited[0] + '-' + splited[-1]
        #print(dataset, type(dataset))
        e1 = float(result_dict[key][subset]['e1'])
        e2 = float(result_dict[key][subset]['e2'])
        data.append([dataset, e1, e2, key, subset])

# 데이터를 Subset 1순위, E1 2순위로 정렬
data_sorted = sorted(data, key=lambda x: (x[0], x[1]))

prev_dataset = ''
for i, row in enumerate(data_sorted):
    if prev_dataset != '' and prev_dataset != row[0]:
        data_sorted[i-1].append(True)
    else:
        data_sorted[i-1].append(False)
    prev_dataset = row[0]

# 정렬된 데이터를 테이블에 추가
for row in data_sorted:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

# 테이블 출력
print(pt)



+---------------------+--------+--------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset       |   E1   |   E2   |                                        Checkpoint                                       |                                 Subset                                 |
+---------------------+--------+--------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN | 152.04 | 113.19 |         MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03        | 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_ORIGINAL-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 178.32 | 128.28 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03 | 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_W

### FIT3D TS_S4710

In [5]:
import prettytable

# PrettyTable 객체 생성 및 필드 설정
pt = prettytable.PrettyTable()
pt.field_names = ['Dataset', 'E1', 'E2', 'Checkpoint', 'Subset']

checkpoint_list = [
    'MB_train_fit3d_gt_cam_no_factor_ts_s4710',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710',
]

# 데이터를 리스트에 수집
data = []
for key in checkpoint_list:
    for subset in result_dict[key].keys():
        if 'STEP' in subset: continue
        if 'TRAIN_UNIV' in subset: continue
        splited = subset.split('-')
        if 'CAM_NO_FACTOR' in splited[-1] or 'CANONICAL' in splited[-1]:
            dataset = splited[0]
        else:
            dataset = splited[0] + '-' + splited[-1]
        #print(dataset, type(dataset))
        e1 = float(result_dict[key][subset]['e1'])
        e2 = float(result_dict[key][subset]['e2'])
        data.append([dataset, e1, e2, key, subset])

# 데이터를 Subset 1순위, E1 2순위로 정렬
data_sorted = sorted(data, key=lambda x: (x[0], x[1]))

prev_dataset = ''
for i, row in enumerate(data_sorted):
    if prev_dataset != '' and prev_dataset != row[0]:
        data_sorted[i-1].append(True)
    else:
        data_sorted[i-1].append(False)
    prev_dataset = row[0]

# 정렬된 데이터를 테이블에 추가
for row in data_sorted:
    pt.add_row([row[0], f'{row[1]:.2f}', f'{row[2]:.2f}', row[3], row[4]], divider=row[-1])

# 테이블 출력
print(pt)



+---------------------+--------+--------+-------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|       Dataset       |   E1   |   E2   |                                         Checkpoint                                        |                                 Subset                                 |
+---------------------+--------+--------+-------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
| 3DHP-TEST_ALL_TRAIN | 159.60 | 116.58 |         MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710        | 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_ORIGINAL-TEST_ALL_TRAIN |
| 3DHP-TEST_ALL_TRAIN | 165.81 | 114.84 | MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710 | 3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANON

### Total

In [ ]:
import prettytable
summary_table = prettytable.PrettyTable()
summary_table.field_names = ['checkpoint', 'subset', 'MPJPE', 'P-MPJPE']
for checkpoint in result_dict.keys():
    for subset in result_dict[checkpoint].keys():
        e1 = result_dict[checkpoint][subset]['e1']
        e2 = result_dict[checkpoint][subset]['e2']
        row = [checkpoint, subset, f'{e1:.2f}', f'{e2:.2f}']
        summary_table.add_row(row)
print(summary_table)

+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                                                                       checkpoint                                                                      |                                          subset                                          | MPJPE  | P-MPJPE |
+-------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+--------+---------+
|                        MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1                       |     H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXE